In [ ]:
from langdetect import detect_langs
import pprint
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, cross_validate, KFold
from sklearn.naive_bayes import BernoulliNB, ComplementNB, MultinomialNB
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, make_scorer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer, TfidfTransformer
import advertools as adv
pp = pprint.PrettyPrinter(indent=4, sort_dicts=False)


#### Loading the data into dataframes and doing light preprocessing 

In [ ]:
def preprocess(data):
    data.text = data.text.apply(lambda x: x.lower())
    data.text = data.text.replace(r"\s+|\\n", " ", regex=True)
    return data


train_data = pd.read_csv("data/train_data.csv")
train_data = preprocess(train_data)
print(train_data.head())



test_data = pd.read_csv("data/test_data.csv")
test_data = preprocess(test_data)
print(test_data.head())

In [ ]:
le = preprocessing.LabelEncoder()
mappings = []
for col in ["language", "label"]:
    train_data[col] = le.fit_transform(train_data[col])
    mappings.append(dict(zip(le.transform(le.classes_), le.classes_)))

print(mappings)

In [ ]:
stop_words = []
for key in ["danish", "german", "dutch", "italian", "spanish"]:
    stop_words+= list(adv.stopwords[key])

# count_vec = TfidfVectorizer(stop_words=stop_words)
count_vec = CountVectorizer(stop_words=stop_words)

train_bow = count_vec.fit_transform(train_data.text)
test_bow = count_vec.transform(test_data.text)

In [ ]:
# language prediction
model = ComplementNB().fit(train_bow, train_data.language)
predicted_langs = model.predict(test_bow)
predicted_langs = pd.Series(predicted_langs, name="language")
test_data = pd.concat([test_data, predicted_langs], axis=1)
test_data.language.head()

In [ ]:
# model = ComplementNB().fit(train_bow, train_data.label)
# predicted_labels = model.predict(test_bow)
# print(predicted_labels)

from collections import defaultdict

train_bows = defaultdict(int)
test_bows = defaultdict(int)
models = defaultdict(int)
predicted_labels = [0] * len(train_data)

for language in mappings[0].keys():
    train_bows[language] = count_vec.fit_transform(train_data[train_data.language == language].text)
    test_bows[language] = count_vec.transform(test_data[test_data.language == language].text)


for language in mappings[0].keys():
    models[language] = ComplementNB().fit(train_bows[language], train_data[train_data.language == language].label)


In [ ]:
predicted_labels = [models[i].predict(test_bows[i]) for i in mappings[0].keys()]
predicted_labels = np.concatenate(predicted_labels)

#### Write the predicted data to a submission file

In [ ]:
final_data = {"id": test_data.index+1, "label": predicted_labels}

submission = pd.DataFrame(data=final_data).set_index("id")
submission = submission.label.apply(lambda x: mappings[1][x])
submission.to_csv("submission_NB.csv")
submission.head()

#### Model checking

In [ ]:
def compare_models(_X, _y, models, _cv=5):
    
    acc = []
    _scoring = {'accuracy': make_scorer(accuracy_score),
                'precision': make_scorer(precision_score, average='macro'),
                'recall': make_scorer(recall_score, average='macro'),
                'f1': make_scorer(f1_score, average='macro')
    }

    kfold = KFold(n_splits=10, shuffle=True, random_state=69420)
    for model in models:
        results = cross_validate(estimator=model,
                                 X=_X,
                                 y=_y,
                                 cv=kfold,
                                 scoring=_scoring,
                                 return_train_score=True)
                                 
        pp.pprint({"Model": type(model).__name__,
                "Training Accuracy scores": results['train_accuracy'],
                "Mean Training Accuracy": results['train_accuracy'].mean()*100,
                "Training Precision scores": results['train_precision'],
                "Mean Training Precision": results['train_precision'].mean(),
                "Training Recall scores": results['train_recall'],
                "Mean Training Recall": results['train_recall'].mean(),
                "Training F1 scores": results['train_f1'],
                "Mean Training F1 Score": results['train_f1'].mean(),
                "Validation Accuracy scores": results['test_accuracy'],
                "Mean Validation Accuracy": results['test_accuracy'].mean()*100,
                "Validation Precision scores": results['test_precision'],
                "Mean Validation Precision": results['test_precision'].mean(),
                "Validation Recall scores": results['test_recall'],
                "Mean Validation Recall": results['test_recall'].mean(),
                "Validation F1 scores": results['test_f1'],
                "Mean Validation F1 Score": results['test_f1'].mean()
                   })
        acc.append(results['test_f1'].mean()*100)
    return acc


models = [ComplementNB(), BernoulliNB(), MultinomialNB()]
print(compare_models(train_bow, train_data.label, models))


In [ ]:
X = train_bow
y = train_data.label
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)

model = ComplementNB().fit(X_train, y_train)
y_pred = model.predict(X_test)
print('Accuracy: ', accuracy_score(y_test, y_pred))
print('F1 score: ', f1_score(y_test, y_pred, average="macro"))